In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Import API key
from config import gkey

In [2]:
# Import cities file into the cities_pd DataFrame

file = pd.read_csv("../Resources/Cities.csv")

cities_pd = pd.DataFrame(file)
cities_pd.head()

,City,State
0,New York City,New York
1,Los Angeles,California
2,Chicago,Illinois
3,Houston,Texas
4,Philadelphia,Pennsylvania


In [3]:
# Add columns for lat, lng, airport name, airport address, airport rating.
# Hint: Be sure to specify "" as the initial value for each column.
cities_pd["Latitude"] = ""
cities_pd["Longitude"] = ""
cities_pd["Airport Name"] = ""
cities_pd["Airport Address"] = ""
cities_pd["Airport Rating"] = ""

cities_pd.head()

,City,State,Latitude,Longitude,Airport Name,Airport Address,Airport Rating
0,New York City,New York,,,,,
1,Los Angeles,California,,,,,
2,Chicago,Illinois,,,,,
3,Houston,Texas,,,,,
4,Philadelphia,Pennsylvania,,,,,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/geocode/json?"

params = {
    "key": gkey
}

In [15]:
# Loop through the cities_pd and get the lat/long for each city
# Hint: `requests.get(target_url).json()`
# Hint: `for index, row in cities_pd.iterrows():`
# Hint: `cities_pd.loc`
for index, row in cities_pd.iterrows():
    params["address"] = row[0] + ", " + row[1]
    
    response = requests.get(base_url, params).json()
    results = response["results"][0]
    
    cities_pd.loc[index,"Latitude"] = results["geometry"]["location"]["lat"]
    cities_pd.loc[index,"Longitude"] = results["geometry"]["location"]["lng"]

cities_pd.head()

,City,State,Latitude,Longitude,Airport Name,Airport Address,Airport Rating
0,New York City,New York,40.7128,-74.006,,,
1,Los Angeles,California,34.0522,-118.244,,,
2,Chicago,Illinois,41.8781,-87.6298,,,
3,Houston,Texas,29.7604,-95.3698,,,
4,Philadelphia,Pennsylvania,39.9526,-75.1652,,,


In [35]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {
    "key": gkey,
    "radius": 50000,
    "types": "airport",
    "keyword": "international airport"
}

In [36]:
# Use the lat/lng we recovered to identify airports
# Hint: Code is similar to above block (i.e. you will need to use cities_pd.iterrows() again)
# Hint: Use the keyword search for "international+airport"
# Hint: Use a try-except to handle airports that are missing ratings
for index, row in cities_pd.iterrows():
    params["location"] = f"{row[2]}, {row[3]}"
    
    response = requests.get(base_url, params).json()
    results = response["results"][0]
    
    try:
        cities_pd.loc[index,"Airport Name"] = results["name"]
        cities_pd.loc[index,"Airport Address"] = results["vicinity"]
        cities_pd.loc[index,"Airport Rating"] = results["rating"]
    except (KeyError, IndexError):
        print("result not found. skipping")
    
cities_pd.head()

,City,State,Latitude,Longitude,Airport Name,Airport Address,Airport Rating
0,New York City,New York,40.7128,-74.006,Newark Liberty International Airport,"3 Brewster Rd, Newark",3.7
1,Los Angeles,California,34.0522,-118.244,Los Angeles International Airport,"1 World Way, Los Angeles",3.8
2,Chicago,Illinois,41.8781,-87.6298,O'Hare International Airport,"10000 W O'Hare Ave, Chicago",4
3,Houston,Texas,29.7604,-95.3698,George Bush Intercontinental Airport,"2800 N Terminal Rd, Houston",4.2
4,Philadelphia,Pennsylvania,39.9526,-75.1652,Philadelphia International Airport,"8000 Essington Ave, Philadelphia",3.8


In [39]:
# Save Data to csv
cities_pd.to_csv("Airport_Output.csv")

# Visualize to confirm airport data appears
cities_pd

,City,State,Latitude,Longitude,Airport Name,Airport Address,Airport Rating
0,New York City,New York,40.7128,-74.006,Newark Liberty International Airport,"3 Brewster Rd, Newark",3.7
1,Los Angeles,California,34.0522,-118.244,Los Angeles International Airport,"1 World Way, Los Angeles",3.8
2,Chicago,Illinois,41.8781,-87.6298,O'Hare International Airport,"10000 W O'Hare Ave, Chicago",4
3,Houston,Texas,29.7604,-95.3698,George Bush Intercontinental Airport,"2800 N Terminal Rd, Houston",4.2
4,Philadelphia,Pennsylvania,39.9526,-75.1652,Philadelphia International Airport,"8000 Essington Ave, Philadelphia",3.8
...,...,...,...,...,...,...,...
95,Fremont,California,37.5485,-121.989,San Francisco International Airport,San Francisco,4.2
96,Baton Rouge,Louisiana,30.4515,-91.1871,Baton Rouge Metropolitan Airport,"9430 Jackie Cochran Dr, Baton Rouge",4.3
97,Richmond,Virginia,37.5407,-77.436,Richmond International Airport,"1 Richard E Byrd Terminal Dr, Richmond",4.4
98,Boise,Idaho,43.615,-116.202,Boise Airport,"3201 W Airport Way #1000, Boise",4.6
